# Benchmark FUSE against GASC

In [ ]:
using Revise
using FUSE
using Plots; gr();
FUSE.logging(Logging.Info);

## Initialize with GASC v1 JSON file (FPPv1.0_aspectRatio3.5_PBpR35.json)

In [ ]:
ini, act = FUSE.case_parameters(:FPP; version=:v1, init_from=:scalars);

### Modify select parameters to better match GASC solution

In [ ]:
ini.equilibrium.R0 = 4.824
ini.equilibrium.ϵ = 1/3.5
ini.equilibrium.κ = 2.19
ini.equilibrium.δ = 0.7
ini.equilibrium.ζ = 0.0

ini.equilibrium.ip = 9.0e6
#ini.material.wall = "Tungsten"
#ini.material.shield = "Steel, Stainless 316"
ini.core_profiles.T_shaping = 2.0
ini.core_profiles.n_shaping = 2.0
ini.core_profiles.greenwald_fraction = 0.95
ini.core_profiles.greenwald_fraction_ped = missing
ini.core_profiles.ne_ped = 1.18e20
ini.core_profiles.impurity = :Kr
ini.core_profiles.zeff = 1.1
ini.pf_active.n_coils_outside = 8


ini.equilibrium.boundary_from = :scalars
ini.equilibrium.xpoints_number = 2

#ini.build.n_first_wall_conformal_layers = 3

#display(ini)

### Modify select actor settings to better match GASC solution

In [ ]:
act.ActorEquilibrium.model = :CHEASE
act.ActorEquilibrium.symmetrize = true
act.ActorCHEASE.free_boundary = true
act.ActorCXbuild.rebuild_wall = true

act.ActorTauenn.T_shaping = 2.0
act.ActorTauenn.confinement_factor = 0.9

act.ActorCosting.model = :Sheffield

#display(act)

### Run FUSE.init

In [ ]:
dd = IMAS.dd()
FUSE.init(dd, ini, act; do_plot=false);
FUSE.digest(dd)

In [ ]:
tf_to_plasma = IMAS.get_build(dd.build, fs=IMAS._hfs_, return_only_one=false, return_index=true)
display(tf_to_plasma)
display(IMAS._offset_)
display(IMAS._negative_offset_)
display(dd.build.layer[14])


## Model entire facility

In [ ]:
#@time FUSE.ActorWholeFacility(dd, act);
#FUSE.digest(dd)